In [ ]:
!pip install keras-tuner -U -qq
import keras_tuner as kt
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript


2022-08-10 13:23:37.884900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 13:23:37.884941: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/rfyahya/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Check the version of TensorFlow Decision Forests
print("Found TensorFlow Decision Forests v" + tfdf.__version__)


Found TensorFlow Decision Forests v0.2.7


In [3]:
import numpy as np
import scipy.stats
from scipy import stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov

def delong_auc(truth, y_pred):
    auc_delong, auc_cov = delong_roc_variance(
    truth,
    y_pred)
    return auc_delong

def delong_auc_cov(truth, y_pred):
    auc_delong, auc_cov = delong_roc_variance(
    truth,
    y_pred)
    return auc_cov

def delong_ci(truth, y_pred):
    auc_delong, auc_cov = delong_roc_variance(
    truth,
    y_pred)

    alpha = .95
    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

    ci = stats.norm.ppf(
        lower_upper_q,
        loc=auc_delong,
        scale=auc_std)

    ci[ci > 1] = 1

    return ci


In [4]:
def Specificity(tn, fp):
    return (tn/(tn+fp))

In [5]:
from tensorflow import keras
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

2022-08-10 13:24:29.902451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 13:24:29.902619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-10 13:24:29.902726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-10 13:24:29.951869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-08-10 13:24:29.952114: W tensorflow/stream_executor/platform/default/dso_loader

In [6]:
test = pd.read_csv("/home/rfyahya/Downloads/test_RAD.csv")
test = test[test.B]
train = pd.read_csv("/home/rfyahya/Downloads/Train_RAD.csv")
val = pd.read_csv("/home/rfyahya/Downloads/Val_RAD.csv")

In [18]:
final_test = final_test[final_test.BraTS21ID != 123]

In [11]:
#hyperparameters tuning: Using grid search, we tune two hyperparameters of the model; the number of samples to split a node and maximum depth of trees.

import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Load your data here

# Define the hyperparameters to tune
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10]
}

# Create a random forest classifier model
rfc = RandomForestClassifier()

# Create a grid search object
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

# Train the model on the grid search object
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and model score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Trial 1000 Complete [00h 00m 03s]
val_accuracy: 0.3617021143436432

Best val_accuracy So Far: 0.7021276354789734
Total elapsed time: 00h 36m 26s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Best hyper-parameters: {'min_examples': 5, 'categorical_algorithm': 'CART', 'max_depth': 35, 'use_hessian_gain': 1, 'shrinkage': 0.27, 'num_candidate_attributes_ratio': 0.7}


In [12]:
# train best model

best_hyper_parameters["use_hessian_gain"] = bool(best_hyper_parameters["use_hessian_gain"])
best_model = tfdf.keras.GradientBoostedTreesModel(**best_hyper_parameters)
best_model.compile(metrics=[METRICS])
best_model.fit(train_ds, verbose=2)


Use /tmp/tmphardcgdb as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'BraTS21ID': <tf.Tensor 'data:0' shape=(None,) dtype=int64>, 'Skewness_Core_T1w': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'Energy_Oedema_T1W': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'GLCM_Contrast_Necrosis_FLAIR': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'GLSZM_Grey_Level_Variance_Enhanced_T1c': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'GLSZM_Low_Grey_Level_Zone_Emphasis_Oedema_T2w': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'original_ngtdm_Busyness_Core_T2w': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>}
Label: Tensor("data_7:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'BraTS21ID': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'Skewness_Core_T1w': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'C

[INFO kernel.cc:813] Start Yggdrasil model training
[INFO kernel.cc:814] Collect training examples
[INFO kernel.cc:422] Number of batches: 1
[INFO kernel.cc:423] Number of examples: 420
[INFO kernel.cc:836] Training dataset:
Number of records: 420
Number of columns: 8

Number of columns by type:
	NUMERICAL: 7 (87.5%)
	CATEGORICAL: 1 (12.5%)

Columns:

NUMERICAL: 7 (87.5%)
	0: "BraTS21ID" NUMERICAL mean:435.324 min:0 max:1010 sd:248.179
	1: "Energy_Oedema_T1W" NUMERICAL mean:9.70047e+10 min:7.11313e+07 max:1.27453e+12 sd:1.76999e+11
	2: "GLCM_Contrast_Necrosis_FLAIR" NUMERICAL mean:589.752 min:0 max:7487.18 sd:920.433
	3: "GLSZM_Grey_Level_Variance_Enhanced_T1c" NUMERICAL mean:16787.3 min:12.7231 max:136993 sd:24510.3
	4: "GLSZM_Low_Grey_Level_Zone_Emphasis_Oedema_T2w" NUMERICAL mean:0.000715942 min:2.39691e-05 max:0.0185257 sd:0.00131884
	5: "Skewness_Core_T1w" NUMERICAL mean:0.127504 min:-4.53399 max:2.02957 sd:0.800462
	6: "original_ngtdm_Busyness_Core_T2w" NUMERICAL mean:0.0295133 m

Model trained in 0:00:00.267066
Compiling model...
Model compiled.


In [ ]:
# At test time, given an unseen subject, the 117 models are all used to predict the
#MGMT methylation status, each predicting either 0 or 1 for the unmethylated
#or methylated group, respectively. The average of the predictions is interpreted
#as the probability of belonging to the methylated group.

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(final_test, label="MGMT_value") 

evaluation = best_model.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

preds = best_model.predict(test_ds, verbose=1)
truth = np.array(list(final_test.MGMT_value))
print(truth.shape)
print(preds.shape)
print("AUC = ", delong_auc(truth, preds.squeeze()))
print("conv = ", delong_auc_cov(truth, preds.squeeze()))
print("ci = ", delong_ci(truth, preds.squeeze()))


In [13]:
Specificity(34, 21)

0.6181818181818182

In [14]:
(2*0.6182* 0.5484)/(0.6182+ 0.5484)

0.5812118635350592